In [396]:
import pandas as pd
import numpy as np
import math

In [362]:
path = 'C:/Users/Rusla/Documents/GitHub/Пушкинская карта - стажировка/Sample/'

In [363]:
users = pd.read_csv(path + 'users_full.csv', sep=';', error_bad_lines=False)

In [364]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879681 entries, 0 to 2879680
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_date         int64  
 1   user_id             object 
 2   user_birth          object 
 3   create_date.1       object 
 4   user_phone_details  object 
 5   buyer_mobile_phone  object 
 6   region              object 
 7   age                 int64  
 8   user_region         float64
dtypes: float64(1), int64(2), object(6)
memory usage: 197.7+ MB


In [365]:
users.sample(3)

,create_date,user_id,user_birth,create_date.1,user_phone_details,buyer_mobile_phone,region,age,user_region
663514,1630497588,44b2d1b19ad87b8794961e38354bc268,2003-01-03,2021-11-19,"Android, 11 (SDK 30)",4e498131d58a719143de0f29d8707e59,Воронежская область,19,36.0
698877,1630736083,3522314cd84248694e99ae5864d6d84a,2005-08-15,NaN,NaN,NaN,Сахалинская область,16,65.0
294653,1630570172,b00190ef01d8644f9c7ec7f1334473e9,1999-11-30,NaN,NaN,NaN,Ставропольский край,22,26.0


In [367]:
users = users.astype({"age": "object", "user_region": "object"})

In [368]:
import json 
expl = [json.loads(line) for line in open(path + 'tickets2.json', 'r', encoding='utf-8', errors = 'ignore')]

In [369]:
tickets = pd.DataFrame(expl)

In [370]:
tickets.sample(3)

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date
165611,1637419076,,353729,1641484800,"сектор:Партер,ряд:7,место:18",7720312951,NaN,6928fa50600e64a65409b07f496d17e6,None,300,,None,1637419076754564997,active,300,10111,NaN
147469,1636179136,,1326163,1636146000,,7810099978,NaN,a7ee765d7c37fc253452c8b1857e827a,None,250,,None,1636179160067590710,active,250,4481,NaN
539472,1637059861,,1422988,1637145000,"Основной зал, Партер, Ряд 4 Место 5",7453098727,NaN,48ae27e92b86dcb6e750986f6f142b0b,None,500.00,Д,None,1637059963164799881,active,500.00,9472,NaN


In [371]:
tickets[tickets['buyer_mobile_phone'].isna()]

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date


In [372]:
tickets.visitor_first_name.nunique() # можно было попробовать пол по именм, но имён у нас всего 11 тысяч из 1,2 млн.

11791

# Добавим данные пользователя в дата сет с купленными билетами

In [373]:
tickets = pd.merge(tickets,users,on=['buyer_mobile_phone'], how='left')

In [374]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854330 entries, 0 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             854330 non-null  int64  
 1   comment                  854330 non-null  object 
 2   session_event_id         854330 non-null  object 
 3   session_date             854330 non-null  int64  
 4   session_params           854330 non-null  object 
 5   owner                    854330 non-null  object 
 6   refund_date              25117 non-null   float64
 7   buyer_mobile_phone       854330 non-null  object 
 8   refund_reason            25117 non-null   object 
 9   payment_amount           854330 non-null  object 
 10  visitor_first_name       854330 non-null  object 
 11  refund_ticket_price      24409 non-null   object 
 12  create_date_x            854330 non-null  int64  
 13  status                   854330 non-null  object 
 14  paym

In [375]:
tickets.user_id.value_counts() # посмотрим как много билетов покупают пользоватли

f038031fea37f04156569ebd7425aae6    90
519e60873d11e8ba3046500f5a63ac7c    80
0bfec9a8f7c33e9f39fa117bb933a30a    63
46ac8e084bdd549553f4aa48ff4e4873    39
dc3b79fc403fa955d4f84ae82cc79da0    38
                                    ..
5d1f8a26f63ac03ff126cebc40d019e7     1
153192910c2c3b36f9822f6b6949a9e1     1
d42218be5e675ed197a355d5f80c8d4d     1
1874773b1703c2712c17fd15a8ac8782     1
abcdef8dddf01e4f8bc2176629587eba     1
Name: user_id, Length: 329522, dtype: int64

In [376]:
tickets.buyer_mobile_phone.value_counts() # посмотрим как много билетов покупают пользоватли

a29e09e61453dd4a978287f9786879b0    65242
da36c3bbd797a083399f9c903af7c65f    15353
6386e9f4ab47751b2f34642e5339b99a     1188
07d06bdd37b69c024f86a5be566f91d4      557
c38b59c8e58401db040b1f773fc59780      370
                                    ...  
d9d90976fbf64dc30a090e37465fa952        1
debf774b7adfc46fdcc901549633192b        1
1c437d4682efaabf77026b6822904df9        1
4a1245fa9325f1bfa60d7ce28d742c29        1
e15a9dee382ee64165a26350ac78d212        1
Name: buyer_mobile_phone, Length: 478364, dtype: int64

In [398]:
tickets[tickets['buyer_mobile_phone'] == '07d06bdd37b69c024f86a5be566f91d4']['owner'].value_counts()

7707079537    557
Name: owner, dtype: int64

In [415]:
tickets[tickets['user_region'] == '2']

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,...,session_organization_id,visit_date,create_date_y,user_id,user_birth,create_date.1,user_phone_details,region,age,user_region


In [413]:
organizations[organizations['Org_region_number'] == 2]['ИНН'].unique()[15]

256017927.0

In [242]:
path2 = 'C:/Users/Rusla/Documents/GitHub/Пушкинская карта - стажировка/Sample/events/'

In [243]:
events = pd.read_csv(path2 + 'events.csv', sep = ';', error_bad_lines=False)

In [244]:
events.sample(3)

,ID,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета
9196,1257973,Спектакль «В сапоге у бабки играл фокстрот»,БУ «Государственный Национальный театр Республ...,https://petrozavodsk.kassy.ru/widget/30-1/even...,NaN
31926,1477355,Спектакль «Солнышко и снежные человечки»,ГБУК ВО «Воронежский государственный театр кук...,http://theatre.bezantrakta.ru/afisha/2021-12-1...,NaN
14634,1299999,Спектакль «Выигрыш»,МУК «Центр культуры и досуга «Мечта»,https://widget.afisha.yandex.ru/w/sessions/tic...,NaN


In [245]:
events[events['ID'] == 1289505]

,ID,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета
13372,1289505,Экспозиции в Петропавловской крепости,Государственный музей истории Санкт-Петербурга,https://spbmuseum.tncloud.ru/event/8AA8E84D96E...,NaN


In [246]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854291 entries, 0 to 854290
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             854291 non-null  int64  
 1   comment                  854291 non-null  object 
 2   session_event_id         854291 non-null  object 
 3   session_date             854291 non-null  int64  
 4   session_params           854291 non-null  object 
 5   owner                    854291 non-null  object 
 6   refund_date              25115 non-null   float64
 7   buyer_mobile_phone       854291 non-null  object 
 8   refund_reason            25115 non-null   object 
 9   payment_amount           854291 non-null  object 
 10  visitor_first_name       854291 non-null  object 
 11  refund_ticket_price      24407 non-null   object 
 12  create_date              854291 non-null  int64  
 13  status                   854291 non-null  object 
 14  paym

In [247]:
events_pushka_accepted_30122021 = pd.read_csv(path2 + 'events_pushka_accepted_30122021.csv', sep = ','
                                              , error_bad_lines=False)

In [248]:
events_pushka_accepted_30122021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47856 entries, 0 to 47855
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   entity._id                    47856 non-null  int64  
 1   entity.name                   47220 non-null  object 
 2   entity.saleLink               47218 non-null  object 
 3   entity.additionalSaleLinks.0  4636 non-null   object 
 4   entity.organization._id       47220 non-null  float64
 5   entity.organization.name      47220 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.2+ MB


In [249]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42154 entries, 0 to 42153
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   ID                                       42154 non-null  int64 
 1   Название события                         42154 non-null  object
 2   Организатор мероприятия                  42154 non-null  object
 3   Ссылка на покупку билета                 42151 non-null  object
 4   Дополнительная ссылка на покупку билета  3817 non-null   object
dtypes: int64(1), object(4)
memory usage: 1.6+ MB


In [250]:
events_2 = events_pushka_accepted_30122021.copy()

In [251]:
events_2 = events_2.rename(
    columns= {'entity._id': 'ID', 'entity.name': 'Название события', 
              'entity.saleLink': 'Ссылка на покупку билета', 
              'entity.additionalSaleLinks.0': 'Дополнительная ссылка на покупку билета',
             'entity.organization.name': 'Организатор мероприятия'}) 

In [252]:
events_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47856 entries, 0 to 47855
Data columns (total 6 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ID                                       47856 non-null  int64  
 1   Название события                         47220 non-null  object 
 2   Ссылка на покупку билета                 47218 non-null  object 
 3   Дополнительная ссылка на покупку билета  4636 non-null   object 
 4   entity.organization._id                  47220 non-null  float64
 5   Организатор мероприятия                  47220 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.2+ MB


In [253]:
events_all = pd.concat([events, events_2])

In [254]:
events_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90010 entries, 0 to 47855
Data columns (total 6 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ID                                       90010 non-null  int64  
 1   Название события                         89374 non-null  object 
 2   Организатор мероприятия                  89374 non-null  object 
 3   Ссылка на покупку билета                 89369 non-null  object 
 4   Дополнительная ссылка на покупку билета  8453 non-null   object 
 5   entity.organization._id                  47220 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ MB


In [255]:
events_all['Название события'].unique()

array(['Спектакль «Касатка»', 'Спектакль «Гамлет»',
       'Спектакль «Горе от ума»', ..., 'Акция «День Викинга»',
       'Спектакль «Отворяя сердце настежь»',
       'Концерт в рамках фестиваля «Звезды НСО РБ»'], dtype=object)

In [256]:
events_all.sample()

,ID,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета,entity.organization._id
5746,1230279,Спектакль «Волшебник Изумрудного города»,ГБУКиИ «Национальный молодежный театр Республи...,https://teatrnmt.ru/afisha/,NaN,NaN


In [257]:
events_all = events_all[events_all['Название события'].notna()]

In [258]:
#создаю столбуц "Событие" куда соберу названия событий, такие как Спектакль, Балет и т.п. из первых слов Названия события
events_all['Событие'] = events_all['Название события'].apply(lambda x: x.split()[0].strip())

In [259]:
events_all.sample(3)

,ID,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета,entity.organization._id,Событие
27889,1313124,Концерт «Plus Испания Ultra»,ГБУК «Государственная академическая капелла Са...,https://capella-spb.ru/ru/afisha/koncerty-sezo...,NaN,6419.0,Концерт
41592,1428995,Оперетта «Зимний вечер в Шамони»,ГБУК г. Москвы «Московский академический Музык...,https://stanmus.ru/shows/zimnij-vecher-v-shamo...,NaN,5312.0,Оперетта
25180,1296377,Концерт «Вечер русской вокальной музыки»,ГАУК МО «Государственный историко-литературный...,https://widget.telegreen.ru/tickets/2c7e07edaf...,NaN,782.0,Концерт


In [260]:
events_all['Событие'].unique() #есть странные названия, посмотрим повнимательнее что у нас получилось

array(['Спектакль', 'Балет', 'Опера', ..., 'Хлебзавод',
       'Трамвайно-пешеходная', '«Афганистан'], dtype=object)

In [261]:
events_all['Событие'].value_counts()

Спектакль                31808
Концерт                  15796
Выставка                  5569
Экскурсия                 5057
Мастер-класс              4239
                         ...  
«Солдатская                  1
«Танцы,                      1
Беломорско-Балтийский        1
Северная                     1
«ВАЛЬС-КОНЦЕРТ»              1
Name: Событие, Length: 2095, dtype: int64

In [262]:
events_all[events_all['Событие'] == '«ВАЛЬС-КОНЦЕРТ»']

,ID,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета,entity.organization._id,Событие
24804,1400515,«ВАЛЬС-КОНЦЕРТ» в рамках Всероссийской акции «...,ГАУК АО «Астраханская государственная филармония»,https://astr.kassir.ru/teatr/astrahanskaya-fil...,NaN,NaN,«ВАЛЬС-КОНЦЕРТ»


In [263]:
events_all['Событие'].nunique()

2095

In [264]:
# Посмотрю по рпавилу Паретто первые 40 Событий
events_all['Событие'].value_counts()[:40].sum()

79681

In [265]:
79681/89374 #первые 40 Событий это 89% от всех событий, всё что не входит в этот список назову "Другое"

0.8915456396714928

In [266]:
drugoe = events_all['Событие'].value_counts()[40:].index

In [267]:
drugoe

Index(['Постоянная', 'Новогодний', 'Открытие', 'Квиз', 'Интеллектуальная',
       'Автобусная', 'Сказка', 'Литературно-музыкальная', 'Театрализованная',
       'Квест-игра',
       ...
       'Вокально-хореографический', 'Литературно-краеведческое', 'Пигмалион',
       '«Здравствуйте,', '«Синяя-синяя', '«Солдатская', '«Танцы,',
       'Беломорско-Балтийский', 'Северная', '«ВАЛЬС-КОНЦЕРТ»'],
      dtype='object', length=2055)

In [268]:
events_all['Событие'] = events_all['Событие'].apply(lambda x: 'Другое' if x in drugoe else x)

In [269]:
events_all['Событие'].value_counts()
#посмотрим что получилось
#Есть похожие События, типа Экспозиция и Экспозиции, похоже по смыслу они входят в выставки, посмотрю внимательно

Спектакль         31808
Концерт           15796
Другое             9693
Выставка           5569
Экскурсия          5057
Мастер-класс       4239
Программа          3681
Лекция             1212
Балет              1182
Экспозиция         1161
Опера              1020
Мюзикл              859
Обзорная            641
Экспозиции          608
Встреча             555
Квест               545
Интерактивная       524
Вечер               392
Тематическая        352
Оперетта            304
Музейный            276
Игра                251
Музыкальная         247
Пешеходная          229
Музейное            228
Гала-концерт        227
Занятие             217
Фестиваль           207
Праздник            205
Концертная          204
Моноспектакль       204
Урок                185
Музыкальный         176
Познавательная      175
Интерактивное       171
Полнокупольная      166
Посещение           165
Творческий          164
Ночь                162
Цикл                159
Творческая          158
Name: Событие, d

In [270]:
events_all[events_all['Событие'] == 'Экспозиция'][:20]

,ID,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета,entity.organization._id,Событие
148,13404,Экспозиция «Природа Удмуртии»,БУК УР «Национальный музей Удмуртской Республи...,https://vmuzey.com/event/ekspoziciya-priroda-u...,NaN,NaN,Экспозиция
1020,134885,Экспозиция Музея истории Томска,МАУ «Музей истории Томска»,https://vmuzey.com/event/osnovnaya-ekspoziciya...,NaN,NaN,Экспозиция
1431,238052,Экспозиция «Портрет семьи Сухановых»,ФГБУК «Музей истории Дальнего Востока имени В....,https://arseniev.tncloud.ru/event/F22EAD127AE3...,NaN,NaN,Экспозиция
1712,339112,Экспозиция «Модель Большого Кремлевского дворц...,ФГБУК «Государственный научно-исследовательски...,https://vmuzey.com/event/model-bolshogo-kremle...,NaN,NaN,Экспозиция
1740,344612,Экспозиция «Мемориальная квартира Анны Ахматовой»,СПб ГБУК «Государственный литературно-мемориал...,https://akhmatova.tncloud.ru/event/97FC3A23AE9...,NaN,NaN,Экспозиция
1758,348988,Экспозиция Мемориального дома-музея И. Левитана,ГБУИО «Плесский музей-заповедник»,https://ples-museum.tncloud.ru/event/D822DB018...,NaN,NaN,Экспозиция
1762,349182,Экспозиция Дома-музея Виктора Васнецова,ФГБУК «Всероссийское музейное объединение «Гос...,https://www.tretyakovgallery.ru/tickets#/buy/e...,NaN,NaN,Экспозиция
1777,352703,Экспозиция «Русское искусство XVI – начала XX ...,ГБУК КК «Краснодарский краевой художественный ...,http://kovalenkomuseum.ru/,NaN,NaN,Экспозиция
1874,382187,Экспозиция Музея бронетанковой техники,ФГБУК «Государственный военно-исторический муз...,https://vmuzey.com/event/ekspoziciya-muzeya-br...,NaN,NaN,Экспозиция
1979,427287,Экспозиция «Ивнянский край в потоке времени»,МКУК «Ивнянский историко-краеведческий музей»,https://bel.cultreg.ru/places/58/ivnyanskii-is...,NaN,NaN,Экспозиция


In [271]:
events_all[events_all['Событие'] == 'Экспозиции'][:20]

,ID,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета,entity.organization._id,Событие
3705,904409,Экспозиции Музея стекла Алексея Зеля,ГБУК КО «Калужский музей изобразительных искус...,https://vmuzey.com/museum/muzey-stekla-aleksey...,NaN,NaN,Экспозиции
6023,1234843,Экспозиции Музея политической истории России,ФГБУК «Государственный музей политической исто...,https://polithistory.ru/visit_us/tickets/,NaN,NaN,Экспозиции
6514,1242499,Экспозиции Музея истории Екатеринбурга,МАУК «Музей истории Екатеринбурга»,https://m-i-e.ru/pushkinkarta,NaN,NaN,Экспозиции
7278,1245803,Экспозиции Российского этнографического музея,ФГБУК «Российский этнографический музей»,https://tickets.ethnomuseum.ru/event/BE56BD176...,NaN,NaN,Экспозиции
7306,1245900,Экспозиции комплекса ладожской крепости,ГБУК ЛО «Музей-заповедник «Старая Ладога»,https://ladogamuseum.tncloud.ru/event/AE8C9315...,NaN,NaN,Экспозиции
7637,1247209,Экспозиции Любытинского краеведческого музея,МБУК «Любытинский краеведческий музей»,https://vmuzey.com/museum/lyubytinskiy-kraeved...,NaN,NaN,Экспозиции
7650,1247258,Экспозиции Музея-заповедника М. Ю. Лермонтова,ГБУК СК «Государственный музей-заповедник М.Ю....,https://vmuzey.com/event/poseshchenie-ekspozic...,NaN,NaN,Экспозиции
7822,1248101,Экспозиции музейно-мемориального комплекса «По...,ГБУК СО «Музейно-мемориальный комплекс «Победа»,https://vmuzey.com/museum/muzeyno-memorialnyy-...,NaN,NaN,Экспозиции
8067,1249519,Экспозиции Музея археологии Москвы,ГБУК «Музейное объединение «Музей Москвы»,https://listim.com/iframe/257/api?gclid=427617...,https://www.mos.ru/afisha/event/146318257/,NaN,Экспозиции
8127,1249761,Экспозиции Оранжерейного комплекса,ГБУК г. Москвы «Государственный историко-архит...,https://www.mos.ru/afisha/event/101217257/,NaN,NaN,Экспозиции


In [272]:
#объединим Экспозиции и Экспозиця в Выставку
expo = ['Экспозиция', 'Экспозиции']
events_all['Событие'] = events_all['Событие'].apply(lambda x: 'Выставка' if x in expo else x)

In [273]:
events_all['Событие'].value_counts()

Спектакль         31808
Концерт           15796
Другое             9693
Выставка           7338
Экскурсия          5057
Мастер-класс       4239
Программа          3681
Лекция             1212
Балет              1182
Опера              1020
Мюзикл              859
Обзорная            641
Встреча             555
Квест               545
Интерактивная       524
Вечер               392
Тематическая        352
Оперетта            304
Музейный            276
Игра                251
Музыкальная         247
Пешеходная          229
Музейное            228
Гала-концерт        227
Занятие             217
Фестиваль           207
Праздник            205
Концертная          204
Моноспектакль       204
Урок                185
Музыкальный         176
Познавательная      175
Интерактивное       171
Полнокупольная      166
Посещение           165
Творческий          164
Ночь                162
Цикл                159
Творческая          158
Name: Событие, dtype: int64

In [274]:
events_all['Организатор мероприятия'].value_counts()

ГАУК ТО «Тюменское концертно-театральное объединение»                      779
ОГБУК «Челябинская государственная филармония»                             649
ГАУК НСО «Новосибирская филармония»                                        574
ФГБУК «Государственный академический Мариинский театр»                     524
ГАУК ТО «Тюменское музейно-просветительское объединение»                   522
                                                                          ... 
ИП Оникиенко Е.Ю.                                                            1
МАУК «Краснобаковский исторический музей»                                    1
ГБУК г. Севастополя «Региональная информационно-библиотечная система»        1
МКУК «Историко-краеведческий музей Светлоярского муниципального района»      1
Зоологический музей Зоологического института РАН                             1
Name: Организатор мероприятия, Length: 3031, dtype: int64

In [275]:
organizations = pd.read_csv(path2 + 'organizations.csv', sep = ';', error_bad_lines=False)

In [276]:
organizations.sample(3)

,ID,Учреждение,Адрес,ИНН,Категория
2814,27213,МБУ «Барнаульский духовой оркестр»,"Алтайский край, г Барнаул",2.224046e+09,Прочее
2695,24052,МКУК ЦКС,"Челябинская обл, г Усть-Катав, ул Комсомольска...",7.401011e+09,Прочее
101,564,МБУК «Кимовский историко-краеведческий музей и...,"Тульская обл, Кимовский р-н, г Кимовск, ул Тол...",7.115022e+09,Музеи и галереи


In [277]:
organizations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3270 entries, 0 to 3269
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          3270 non-null   int64  
 1   Учреждение  3270 non-null   object 
 2   Адрес       3270 non-null   object 
 3   ИНН         3260 non-null   float64
 4   Категория   3232 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 127.9+ KB


In [278]:
organizations = organizations[organizations['ИНН'].notna()]

In [279]:
len(str(organizations[organizations['ID'] == 4576]['ИНН'].values[0]))

11

In [280]:
organizations['Org_region_number'] = organizations['ИНН'].apply(lambda x:
                                                               int(str(x)[:2]) 
                                                                if len(str(x)) == 12
                                                               else int(str(x)[:1]))

In [281]:
organizations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3260 entries, 0 to 3269
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 3260 non-null   int64  
 1   Учреждение         3260 non-null   object 
 2   Адрес              3260 non-null   object 
 3   ИНН                3260 non-null   float64
 4   Категория          3222 non-null   object 
 5   Org_region_number  3260 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 178.3+ KB


In [282]:
organizations.sample(3)

,ID,Учреждение,Адрес,ИНН,Категория,Org_region_number
2748,25678,МАУК «Дворец культуры «Юбилейный»,"Удмуртская Респ, г Воткинск, ул Мира, д 19",1.828013e+09,Дворцы культуры и клубы,18
543,3967,МБУ «Шимановский краеведческий музей»,"Амурская обл, г Шимановск, ул Первомайская, д 10",2.809003e+09,Музеи и галереи,28
214,2094,ФГБУКИ «Военно-Медицинский музей»,"г Санкт-Петербург, Лазаретный пер, д 2",7.838041e+09,Музеи и галереи,78


In [283]:
organizations = organizations.rename(
    columns= {'Учреждение': 'Организатор мероприятия',}) 

In [284]:
organizations[organizations['ID'] == 6]

,ID,Организатор мероприятия,Адрес,ИНН,Категория,Org_region_number
1,6,ОГАУК «Драматический театр имени М. С. Щепкина»,"Белгородская обл, г Белгород, Соборная пл, д 1",3.124011e+09,Театры,31


In [285]:
#events_all = pd.concat([events_all, organizations['ID'], organizations['Org_region_number']],ignore_index = True, 
                       #axis = 1, keys = 'Организатор мероприятия')
events_all = pd.merge(events_all,organizations,on=['Организатор мероприятия'], how='left')

In [286]:
events_all.sample(3)

,ID_x,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета,entity.organization._id,Событие,ID_y,Адрес,ИНН,Категория,Org_region_number
24621,1398579,«ВЕСЁЛЫЕ ОСЬМИНОЖКИ»,МБУК МТК и А «Буратино»,https://magn.kassy.ru/events/teatr/2-22886/sec...,NaN,NaN,Другое,7832.0,"Челябинская обл, г Магнитогорск, ул Бориса Руч...",7.414002e+09,Театры,74.0
54879,1246864,Мюзикл «Золушка»,ГАУК г.Москвы «КЦ «Зеленоград»,https://dkzelenograd.ru/afisha/detyam/zolushka,NaN,6843.0,Мюзикл,6843.0,"г Москва, г Зеленоград, Центральная пл, д 1",7.735071e+09,Дворцы культуры и клубы,77.0
1712,339112,Экспозиция «Модель Большого Кремлевского дворц...,ФГБУК «Государственный научно-исследовательски...,https://vmuzey.com/event/model-bolshogo-kremle...,NaN,NaN,Выставка,156.0,"г Москва, ул Воздвиженка, д 5/25",7.704061e+09,Музеи и галереи,77.0


In [287]:
events_all = events_all.rename(
    columns= {'ID_x': 'session_event_id', 'ID_y': 'session_organization_id'}) 

In [288]:
events_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89670 entries, 0 to 89669
Data columns (total 12 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   session_event_id                         89670 non-null  int64  
 1   Название события                         89670 non-null  object 
 2   Организатор мероприятия                  89670 non-null  object 
 3   Ссылка на покупку билета                 89665 non-null  object 
 4   Дополнительная ссылка на покупку билета  8456 non-null   object 
 5   entity.organization._id                  47334 non-null  float64
 6   Событие                                  89670 non-null  object 
 7   session_organization_id                  88733 non-null  float64
 8   Адрес                                    88733 non-null  object 
 9   ИНН                                      88733 non-null  float64
 10  Категория                                86448

In [289]:
events_all = events_all.astype({"session_event_id": "object", "session_organization_id": "object",
                               "Org_region_number": "object"})

In [299]:
events_all.sample(3)

,session_event_id,Название события,Организатор мероприятия,Ссылка на покупку билета,Дополнительная ссылка на покупку билета,entity.organization._id,Событие,session_organization_id,Адрес,ИНН,Категория,Org_region_number
79439,1383446,Концерт симфонической музыки,ГБУКиИ «Национальный симфонический оркестр Рес...,https://iframeab-pre2273.intickets.ru/seance/1...,NaN,7220.0,Концерт,7220,"Респ Башкортостан, г Уфа, ул Ленина, д 50",2.740540e+08,Прочее,2
51309,1232958,Спектакль «Страшный суп»,ГАУК СО «Свердловский государственный академич...,https://uraldrama.ticketland.ru/buy-ticket/tea...,NaN,5645.0,Спектакль,5645,"Свердловская обл, г Екатеринбург, Октябрьская ...",6.658013e+09,Театры,66
17223,1324423,Экспозиция «Этно. Образ. Миф. Знак»,МБУК «Усинский музейно-выставочный центр «Вортас»,https://vmuzey.com/event/vystavka-zhivopisi,NaN,NaN,Выставка,10543,"Респ Коми, г Усинск, ул Парковая, д 4А",1.106024e+09,Музеи и галереи,11


In [291]:
tickets.sample(3) #как видно из этой вырезки у части мероприятий нет id организотора, надo добавить

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date
814546,1637301133,,1344566,1639922400,"Партер, ряд 7, место 14",7714836675,NaN,94f160bd478dffb9e92a7e8753a427ed,None,500,,None,1637304304198542692,active,500,6407,NaN
187312,1636796207000,,1316770,1638028800000,Площадка: РАМТ,9705121040,1.637244e+12,f662632c680b2729fac28f4d7e37674b,Меня не будет в городе,RUB 1500.00,Татьяна,,1636796237141980443,refunded,RUB 1500.00,,NaN
777190,1637940899,,1470162,1638959400,"Зал Музыкального училища , 9 ряд, 5 место",7107520929,NaN,143efc554ea1a37af509933b969eacc1,None,300.00,,None,1637941224140234102,active,300.00,12684,NaN


In [344]:
events_all[events_all['session_organization_id'].isna()]['Организатор мероприятия'].value_counts()
#как видим есть организаторы у которых нет id у части мероприятий - 

СПБ ГБУК «Санкт-Петербургский Государственный Музыкально-Драматический Театр «БУФФ»    113
ГАУК «Курганское областное музейное объединение»                                        93
КГАУК «Хабаровский краевой музыкальный театр»                                           69
МБУК «Березниковский историко-художественный музей им. И. Ф. Коновалова»                57
Кемеровский театр для детей и молодёжи                                                  51
                                                                                      ... 
АУК УР «Государственный цирк Удмуртии»                                                   1
МБУК «Дворец металлургов»                                                                1
Тестовое учреждение (встреча)                                                            1
БУ РА «Национальный музей им. А. В. Анохина»                                             1
ГКУК «Ансамбль Терских Казаков»                                                          1

In [315]:
id_null_org = events_all[events_all['session_organization_id'].isna()]['Организатор мероприятия'].value_counts().index

In [338]:
id_null_org_map = {}

for i in id_null_org:
    j = events_all[events_all['Организатор мероприятия'] == i]['entity.organization._id'].unique()
    for x in j:
        if math.isnan(x) != True:
            id_null_org_map[i] = int(x)

In [342]:
len(id_null_org_map)

67

In [345]:
events_all_id_null = events_all[events_all['session_organization_id'].isna()]

In [346]:
events_all_id_null['session_organization_id'] = events_all_id_null['Организатор мероприятия'].map(id_null_org_map)

C:\Users\Rusla\AppData\Local\Temp/ipykernel_30932/942977657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_all_id_null['session_organization_id'] = events_all_id_null['Организатор мероприятия'].map(id_null_org_map)


In [347]:
events_all_id_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 937 entries, 896 to 89348
Data columns (total 12 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   session_event_id                         937 non-null    object 
 1   Название события                         937 non-null    object 
 2   Организатор мероприятия                  937 non-null    object 
 3   Ссылка на покупку билета                 936 non-null    object 
 4   Дополнительная ссылка на покупку билета  37 non-null     object 
 5   entity.organization._id                  732 non-null    float64
 6   Событие                                  937 non-null    object 
 7   session_organization_id                  925 non-null    float64
 8   Адрес                                    0 non-null      object 
 9   ИНН                                      0 non-null      float64
 10  Категория                                0 non